### 資料集簡介
<p>欄位說明:</p>
<p>FileID: 檔案識別ID</p>
<p>CustomerID: 使用者裝置識別ID</p>
<p>QueryTs: 該筆資料發生時間</p>
<p>ProductID: 使用者裝置的產品代碼</p>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
plt.rcParams['font.family']='SimHei' #顯示中文

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
# Load in the train datasets
data_dir = "/data/examples/trend/data/"
train = pd.read_csv(data_dir + 'training-set.csv', encoding = "utf-8", header=None)
test = pd.read_csv(data_dir + 'testing-set.csv', encoding = "utf-8", header=None)

In [4]:
#query_log裡面被官方排除的 FileID
train_exc = pd.read_csv(data_dir + 'exception/exception_train.txt', encoding = "utf-8", header=None)
test_exc = pd.read_csv(data_dir + 'exception/exception_testing.txt', encoding = "utf-8", header=None)

In [5]:
test_exc.head(2)

,0
0,4eedf630f7160dafea969e5d57239d31
1,8e1c12f42b705cb465bab88225d03c81


In [38]:
# training set - label: 0:非惡意程式, 1:惡意程式
train.columns=['FileID','label']
train.head(2)

,FileID,label
0,0000e2398b12121a85166fed5fe2a3da,0
1,0001fe8dce14ce099aa6ca8ea5026ea7,0


In [39]:
# testing set - AUC: Area Under ROC Curve
test.columns=['FileID','AUC']
test.head(2)

,FileID,AUC
0,00008c73ee43c15b16c26b26398c1577,0.5
1,0002ded3a0b54f2ffdab0ca77a5ce2b6,0.5


In [8]:
train_exc.columns = ['FileID']
test_exc.columns = ['FileID']
train_exc.head(2)
test_exc.head(2)

,FileID
0,4eedf630f7160dafea969e5d57239d31
1,8e1c12f42b705cb465bab88225d03c81


In [9]:
#確認排除的FileID在training set裡面找不到
for item in train_exc:
    print(train[train['FileID']==item])

Empty DataFrame
Columns: [FileID, label]
Index: []


In [10]:
#取0301當天的query log來查看
log_files = os.listdir(data_dir+"query_log")
query_logs = []
for log_file in log_files:
    query_log = pd.read_csv(data_dir + 'query_log/' + log_file, encoding = "utf-8", header=None)
    query_log.columns=['FileID','CustomerID','QueryTs','ProductID']
    query_log['times'] = 1
    query_logs.append(query_log)

In [11]:
querys = pd.concat(query_logs)

In [12]:
querys = querys.sort_values(by=['FileID','CustomerID','ProductID','QueryTs'])

In [13]:
file_product_mean = querys.groupby(["FileID", "ProductID"])["QueryTs"].mean()
file_product_std = querys.groupby(["FileID", "ProductID"])["QueryTs"].std()

In [16]:
querys.head(2)

,FileID,CustomerID,QueryTs,ProductID,times
229005,00008c73ee43c15b16c26b26398c1577,033e91e730e176cd2ac81dc48a8f5088,1494828718,7acab3,1
442077,00008c73ee43c15b16c26b26398c1577,033e91e730e176cd2ac81dc48a8f5088,1494860581,7acab3,1


In [18]:
querys.describe()

,QueryTs,times
count,8.327311e+07,83273110.0
mean,1.492394e+09,1.0
std,2.138802e+06,0.0
min,1.488326e+09,1.0
25%,1.490234e+09,1.0
50%,1.492291e+09,1.0
75%,1.494531e+09,1.0
max,1.496275e+09,1.0


In [19]:
querys.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83273110 entries, 229005 to 47818
Data columns (total 5 columns):
FileID        object
CustomerID    object
QueryTs       int64
ProductID     object
times         int64
dtypes: int64(2), object(3)
memory usage: 20.2 GB


### 探索式資料分析

In [20]:
querys.columns

Index(['FileID', 'CustomerID', 'QueryTs', 'ProductID', 'times'], dtype='object')

In [21]:
len(list(set(querys["CustomerID"])))

5539312

In [22]:
len(list(set(querys["ProductID"])))

32

### 樞杻分析

In [23]:
query_pivot_productid = querys.pivot_table(values='times',index=['FileID'],columns='ProductID',aggfunc='sum')
# query_pivot_customerid = querys.pivot_table(values='times',index=['FileID'],columns='CustomerID',aggfunc='sum')

In [41]:
test.columns

Index(['FileID', 'AUC'], dtype='object')

In [42]:
train_query_pivot_productid = query_pivot_productid.ix[train["FileID"]]
test_query_pivot_productid = query_pivot_productid.ix[test["FileID"]]
train_query_pivot_productid = train_query_pivot_productid.fillna(0)
test_query_pivot_productid = test_query_pivot_productid.fillna(0)

## 額外 features

In [43]:
querys["Day"] = querys["QueryTs"] // (3600 * 24)

In [44]:
df_FileID_Day = querys.pivot_table(values='times', index='FileID', columns='Day', aggfunc='sum')
df_FileID_Day = df_FileID_Day.fillna(0)

# FileID vs Day 的各種 features
df_FileID_Day_features = pd.DataFrame(columns=['Total(Day)', 'Max(Day)', 'Min(Day)', 'Mean(Day)', 'Mean(Nonzero Day)', 'Std(Day)'])
df_FileID_Day_features['Total(Day)'] = df_FileID_Day.sum(axis=1)
df_FileID_Day_features['Max(Day)'] = df_FileID_Day.max(axis=1)
df_FileID_Day_features['Min(Day)'] = df_FileID_Day.min(axis=1)
df_FileID_Day_features['Mean(Day)'] = df_FileID_Day.mean(axis=1)
df_FileID_Day_features['Mean(Nonzero Day)'] = df_FileID_Day_features['Total(Day)'] / (df_FileID_Day > 0).sum(axis=1)
df_FileID_Day_features['Std(Day)'] = df_FileID_Day.std(axis=1)
df_FileID_Day_features['Std(Nonzero Day)'] = (df_FileID_Day[df_FileID_Day > 0]).std(axis=1)
df_FileID_Day_features = df_FileID_Day_features.fillna(0)

# FileID vs Product 的各種 features
df_FileID_ProductID = querys.pivot_table(values='times', index='FileID', columns='ProductID', aggfunc='sum')
df_FileID_ProductID = df_FileID_ProductID.fillna(0)
df_FileID_ProductID_features = pd.DataFrame(columns=['Total(ProductID)', 'Max(ProductID)', 'Min(ProductID)', 'Mean(ProductID)', 'Std(ProductID)'])
df_FileID_ProductID_features['Total(ProductID)'] = df_FileID_ProductID.sum(axis=1)
df_FileID_ProductID_features['Max(ProductID)'] = df_FileID_ProductID.max(axis=1)
df_FileID_ProductID_features['Min(ProductID)'] = df_FileID_ProductID.min(axis=1)
df_FileID_ProductID_features['Mean(ProductID)'] = df_FileID_ProductID.mean(axis=1)
df_FileID_ProductID_features['Std(ProductID)'] = df_FileID_ProductID.std(axis=1)
df_FileID_ProductID_features = df_FileID_ProductID_features.fillna(0)

In [45]:
train_df = train_query_pivot_productid.join(df_FileID_Day_features)
train_df = train_df.join(df_FileID_ProductID_features)
test_df = test_query_pivot_productid.join(df_FileID_Day_features)
test_df = test_df.join(df_FileID_ProductID_features)

## 基本前處理

### 正規化, 標準縮放

In [46]:
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [47]:
# ss = StandardScaler()
train_standard = ss.fit_transform(train_query_pivot_productid)
test_standard = ss.transform(test_query_pivot_productid)
# nl = Normalizer()
# train_norm = nl.fit_transform(train_df)
# test_norm = nl.transform(test_df)

In [49]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier
import xgboost as xgb

### Ensembling

In [50]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1) 
    #Now we do the predictions for cloned models and average them
    def predict_proba(self, X):
        probs = np.mean([ model.predict_proba(X) for model in self.models_ ], axis=0)
        return probs

### Validation Evaluation

In [124]:
train_shuffled_indices = np.arange(train_norm.shape[0])
np.random.shuffle(train_shuffled_indices)
y = train['label'].values
valid_ratio = 0.1
split_index = int(len(train_shuffled_indices)*(1- valid_ratio))

X_train = train_norm[train_shuffled_indices[0:split_index]]
X_valid = train_norm[train_shuffled_indices[split_index: len(train_shuffled_indices)]]
y_train = y[train_shuffled_indices[0:split_index]]
y_valid = y[train_shuffled_indices[split_index: len(train_shuffled_indices)]]

In [187]:
Logit = LogisticRegression(
    penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
    intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear',
    max_iter=1000, multi_class='ovr', verbose=1, warm_start=False, n_jobs=1
)
GBoost = GradientBoostingClassifier(
    n_estimators=4000, learning_rate=0.05, max_depth=5, max_features='sqrt',
    min_samples_leaf=15, min_samples_split=10, loss='deviance', random_state =5, verbose=1
)
RandomForest = RandomForestClassifier(
    n_estimators=10, criterion="gini", max_depth=4, min_samples_split=1.0, min_samples_leaf=2,
    min_weight_fraction_leaf=0.0, max_features="auto", max_leaf_nodes=None, min_impurity_decrease=0.0,
    min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=1, warm_start=False,
    class_weight="balanced"
)
XGB = xgb.XGBClassifier(
    max_depth=5,learning_rate=0.1, n_estimators=1500, silent=False, objective='binary:logistic', booster='gbtree', 
    n_jobs=4, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, 
    colsample_bylevel=1, reg_alpha=0.0, reg_lambda=1.2, scale_pos_weight=1, verbose_eval=True
)

In [188]:
# averaged_models = AveragingModels(models = (Logit, GBoost, RandomForest))
# averaged_models.fit(X_train, y_train)

In [189]:
# y_valid_score = averaged_models.predict_proba(X_valid)

In [183]:
# RandomForest.fit(X_train, y_train)
# y_valid_score = RandomForest.predict_proba(X_valid)

In [190]:
XGB.fit(X_train, y_train, eval_metric="auc", eval_set=[(X_valid, y_valid)], early_stopping_rounds=100)
y_valid_score = XGB.predict_proba(X_valid)

[0]	validation_0-auc:0.835247
Will train until validation_0-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.85069
[2]	validation_0-auc:0.848733
[3]	validation_0-auc:0.854218
[4]	validation_0-auc:0.852601
[5]	validation_0-auc:0.856432
[6]	validation_0-auc:0.859208
[7]	validation_0-auc:0.862925
[8]	validation_0-auc:0.864324
[9]	validation_0-auc:0.865823
[10]	validation_0-auc:0.867302
[11]	validation_0-auc:0.86774
[12]	validation_0-auc:0.86844
[13]	validation_0-auc:0.869616
[14]	validation_0-auc:0.871599
[15]	validation_0-auc:0.873342
[16]	validation_0-auc:0.873235
[17]	validation_0-auc:0.873775
[18]	validation_0-auc:0.874545
[19]	validation_0-auc:0.874647
[20]	validation_0-auc:0.87665
[21]	validation_0-auc:0.877948
[22]	validation_0-auc:0.878887
[23]	validation_0-auc:0.880377
[24]	validation_0-auc:0.880635
[25]	validation_0-auc:0.881581
[26]	validation_0-auc:0.883202
[27]	validation_0-auc:0.883537
[28]	validation_0-auc:0.884649
[29]	validation_0-auc:0.885132
[30]	validation_0-a

In [191]:
roc_auc_score(y_train, XGB.predict_proba(X_train)[:,1]) 

0.96053677161580464

In [192]:
roc_auc_score(y_valid, y_valid_score[:,1]) 

0.90729097805856362

### 計算測資結果

In [193]:
X_test = test_norm.copy()
y_test_score = XGB.predict_proba(X_test)
test_out = pd.DataFrame.from_dict({
    "FileID": test["FileID"],
    "Probability": y_test_score[:,1]
})
test_out.to_csv('submission.csv', index=False)

### 聚合函數
count（個數）, sum（加總）, mean（平均）, median（中位數）, std（標準差）, var（變異數）, first（第一個非NA）, last（最後一個非NA）

In [113]:
# querys.groupby(['FileID','CustomerID','ProductID'])[['times']].sum()

### 其它會用到的工具

In [114]:
#AUC計算範例
import numpy as np
from sklearn import metrics
y = np.array([1, 1, 2, 2])
pred = np.array([0.9, 0.1, 0.7, 0.9])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
metrics.auc(fpr, tpr)


0.625

In [ ]:
#timestamp轉換
import datetime
print(
    datetime.datetime.fromtimestamp(
        int("1488326402")
    ).strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
querys.pivot_table(values='QueryTs', index='FileID', columns='ProductID').head()

In [ ]:
time_list = querys.loc[(querys['FileID']=='0000e2398b12121a85166fed5fe2a3da') & (querys['ProductID']=='055649')][['QueryTs']]

In [ ]:
time_list.values